# Data Preprocessing / Preprocesamiento de Datos

In [3]:
import os
import librosa
import numpy as np
import pandas as pd

## LDA, GDA

In [6]:
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    f0, voiced_flag, voiced_probs = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    
    features = {
        'meanfreq': np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
        'sd': np.std(y),
        'median': np.median(y),
        'Q25': np.percentile(y, 25),
        'Q75': np.percentile(y, 75),
        'IQR': np.percentile(y, 75) - np.percentile(y, 25),
        'skew': pd.Series(y).skew(),
        'kurt': pd.Series(y).kurt(),
        'sp.ent': np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)),
        'sfm': np.mean(librosa.feature.spectral_flatness(y=y)),
        'mode': np.argmax(np.bincount(y.astype(int))),
        'centroid': np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
        'meanfun': np.mean(librosa.feature.rms(y=y)),
        'minfun': np.min(librosa.feature.rms(y=y)),
        'maxfun': np.max(librosa.feature.rms(y=y)),
        'meandom': np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)),
        'mindom': np.min(librosa.feature.spectral_rolloff(y=y, sr=sr)),
        'maxdom': np.max(librosa.feature.spectral_rolloff(y=y, sr=sr)),
        'dfrange': np.max(librosa.feature.spectral_rolloff(y=y, sr=sr)) - np.min(librosa.feature.spectral_rolloff(y=y, sr=sr)),
        'modindx': np.std(librosa.feature.spectral_rolloff(y=y, sr=sr)) / np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)),
        'mean_f0': np.nanmean(f0),
        'min_f0': np.nanmin(f0),
        'max_f0': np.nanmax(f0)
    }
    
    for i in range(13):
        features[f'mfcc_{i+1}'] = np.mean(mfccs[i, :])
    
    return features

# Cargar el archivo Excel
excel_path = "raw/BVC_Voice_Bio_Public.xlsx"
df_labels = pd.read_excel(excel_path)
id_to_sex = dict(zip(df_labels['New_ID'], df_labels['Sex']))

dataset = []

# Procesar archivos de audio
for file in os.listdir("raw/audio"):
    if file.endswith(".wav"):
        file_id = int(file.split('_')[2])  # Extraer el ID antes del guion bajo
        if file_id in id_to_sex:
            file_path = os.path.join("raw/audio", file)
            features = extract_features(file_path)
            features['label'] = id_to_sex[file_id]
            dataset.append(features)

# Convertir a DataFrame y guardar
df_dataset = pd.DataFrame(dataset)
df_dataset.to_csv("dataset_LDA-GDA.csv", index=False)
print("Dataset guardado en data/dataset_LDA-GDA.csv")


Dataset guardado en data/dataset_LDA-GDA.csv
